In [4]:
!pip install -Uqq fastai

In [5]:
import fastai
fastai.__version__

In [6]:
import time
import numpy as np
import pandas as pd
from fastcore.all import *
from fastai.vision.all import *
from fastdownload import download_url

In [7]:
import io
import requests
import matplotlib.pyplot as plt 

In [8]:
def display_image_from_url(url):
    data = requests.get(url).content
    img = fastai.vision.all.Image.open(io.BytesIO(data))
    img = img.to_thumb(256,256)
    plt.imshow(img)
    plt.show()

In [9]:
def gallery(img_paths, max=6, max_cols=3, figsize=(8, 8)):
    img_paths = img_paths[0:max]
    fig = plt.figure(figsize=figsize)
    columns = min(max_cols, len(img_paths))
    rows = len(img_paths) // columns + 1
    for i, path in enumerate(img_paths):
        img = fastai.vision.all.Image.open(path)
        img = img.to_thumb(256,256)
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(img)
    plt.show()

In [10]:
def search_images(term, max_images=200):
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    urls,data = set(),{'next':1}
    while len(urls)<max_images and 'next' in data:
        data = urljson(requestUrl,data=params)
        urls.update(L(data['results']).itemgot('image'))
        requestUrl = url + data['next']
        time.sleep(0.2)
    return L(urls)[:max_images]

In [34]:
search_terms = [
    "fantail",
    "kaka",
    "kereru",
    "morepork",
    "tui",
]

In [12]:
searches = {
    term: search_images(term+" bird photo")
    for term in search_terms
}

In [14]:
path = Path('bird_classification')
for bird,urls in searches.items():
    dest = (path/bird)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=urls)
    resize_images(path/bird, max_size=400, dest=path/bird)

In [15]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [16]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

In [17]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [18]:
dest = 'fantail.jpg'
download_url("https://nzbirdsonline.org.nz/sites/all/files/IMG_0690.JPG", dest, show_progress=true)

In [19]:
result = learn.predict(PILImage.create('fantail.jpg'))

In [20]:
result

In [44]:
url = "https://nzbirdsonline.org.nz/sites/all/files/1200543IMG_0709nzbo.jpg"
dest = 'test_image.jpg'
download_url(url, dest, show_progress=true)

img = Image.open(dest)
plt.imshow(img)
plt.show()

pred, _, probs = learn.predict(PILImage.create(dest))
print(pred)
print("-"*25)
for name,p in zip(search_terms, probs):
    print(f"{name:10} {p:0.08f}")